In [1]:
from ultralytics import YOLO
model=YOLO("yolov8m.pt")

In [ ]:
model.train(data="/mnt/c/Users/drenergydrink/Desktop/Document-and-Record-Management/YOLOv8/data/data.yaml",epochs=80)

In [4]:
yolo = YOLO("/mnt/c/Users/drenergydrink/Desktop/Document-and-Record-Management/pretrained models/YOLOv8m-Driving License Best.pt"); yolo.predict(source="/mnt/c/Users/drenergydrink/Desktop/Document-and-Record-Management/YOLOv8/mainali.jpg", save=True)



image 1/1 /mnt/c/Users/drenergydrink/Desktop/Document-and-Record-Management/YOLOv8/mainali.jpg: 480x640 1 address, 1 blood_group, 1 citizenship_number, 1 contact_number, 1 dob, 1 document_type, 1 father_name, 1 license_category, 1 license_number, 2 names, 22.2ms
Speed: 4.3ms preprocess, 22.2ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict2


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'address', 1: 'blood_group', 2: 'citizenship_number', 3: 'contact_number', 4: 'dob', 5: 'document_type', 6: 'father_name', 7: 'license_category', 8: 'license_number', 9: 'name'}
 obb: None
 orig_img: array([[[10, 16, 21],
         [10, 16, 21],
         [10, 16, 21],
         ...,
         [64, 76, 88],
         [61, 73, 85],
         [58, 70, 82]],
 
        [[10, 16, 21],
         [10, 16, 21],
         [10, 16, 21],
         ...,
         [63, 75, 87],
         [62, 74, 86],
         [61, 73, 85]],
 
        [[ 9, 15, 20],
         [10, 16, 21],
         [10, 16, 21],
         ...,
         [63, 75, 87],
         [63, 75, 87],
         [61, 73, 85]],
 
        ...,
 
        [[10, 14, 19],
         [ 9, 13, 18],
         [ 9, 13, 18],
         ...,
         [ 4,  6,  7],
         [ 5,  7,  8],
         [ 5,  7,  8]],
 
        [[10, 